In [1]:
CRITERIA = {
    "Squat": [
        {
            "name": "knees",
            "description": "Knees reach 90 degrees",
            "long_description": "The angle between your hip, knee, and ankle must reach 90 degrees. Your thigh should be parallel to the ground.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 90,
            "requirement": "require",
            "feedback_fail": "Lower your squat to at least 90 degrees!",
            "feedback_pass": "Great depth! Keep it up!",
        },
        {
            "name": "shoulders",
            "description": "Shoulders remain level",
            "long_description": "Your shoulders should be at the same height during the squat to avoid tilting.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders level while squatting!",
            "feedback_pass": "Nice! Your shoulders stayed balanced!",
        },
        {
            "name": "torso",
            "description": "Torso remains upright",
            "long_description": "Your back should be straight, and eyes forward.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 70,
            "requirement": "avoid",
            "feedback_fail": "Straighten your back and look forward!",
            "feedback_pass": "Great posture!",
        },
    ],
    "Push-Up": [
        {
            "name": "level-shoulders",
            "description": "Shoulders stay level",
            "long_description": "Your shoulders should not tilt past 25 degrees.",
            "condition": lambda angles: angles["left_shoulderright_shoulder"] < 25,
            "requirement": "avoid",
            "feedback_fail": "Keep your shoulders steady during push-ups!",
            "feedback_pass": "Good control over your shoulders!",
        },
        {
            "name": "tricep",
            "description": "Arms fully extend",
            "long_description": "At the top position, your triceps should form a 90-degree angle with the ground.",
            "condition": lambda angles: angles["right_shoulderright_elbow"] >= 80,
            "requirement": "require",
            "feedback_fail": "Extend your arms fully at the top!",
            "feedback_pass": "Good full extension!",
        },
    ],
    "Sit-Up": [
        {
            "name": "up-position",
            "description": "Back gets perpendicular to ground",
            "long_description": "When you are in the 'up' position, your back should be at least 75 degrees.",
            "condition": lambda angles: angles["right_shoulderright_hip"] >= 75,
            "requirement": "require",
            "feedback_fail": "Sit up higher to reach the correct position!",
            "feedback_pass": "Good sit-up position!",
        },
        {
            "name": "down-position",
            "description": "Back gets horizontal to ground",
            "long_description": "When you are in the down position, your back should be flat with the ground, including your neck.",
            "condition": lambda angles: angles["left_shoulderleft_hip"] <= 5,
            "requirement": "require",
            "feedback_fail": "Lie back fully to complete the rep!",
            "feedback_pass": "Nice full range of motion!",
        },
        {
            "name": "bent legs",
            "description": "Legs stay bent",
            "long_description": "During a sit-up, your legs should stay bent and not move excessively.",
            "condition": lambda angles: angles["right_hipright_knee"] <= 10,
            "requirement": "avoid",
            "feedback_fail": "Keep your legs stable and bent throughout the movement!",
            "feedback_pass": "Great job keeping your legs steady!",
        },
    ]
}


In [41]:
import numpy as np
import math

def get_bending_angle(hip: np.ndarray, knee: np.ndarray, ankle: np.ndarray) -> float:
    """Calculate the bending angle of the knee using 2D coordinates (MoveNet format).
    
    Args:
        hip (np.ndarray): (x, y, confidence) of the hip keypoint.
        knee (np.ndarray): (x, y, confidence) of the knee keypoint.
        ankle (np.ndarray): (x, y, confidence) of the ankle keypoint.

    Returns:
        float: Knee bending angle in degrees.
    """
    # Femur (thigh)
    vector1 = (knee[0] - hip[0], knee[1] - hip[1])
    # Tibia (shin)
    vector2 = (ankle[0] - knee[0], ankle[1] - knee[1])

    # Compute dot product
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]

    # Compute magnitudes
    magnitude1 = math.sqrt(vector1[0]**2 + vector1[1]**2)
    magnitude2 = math.sqrt(vector2[0]**2 + vector2[1]**2)

    # Avoid division by zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0

    # Calculate angle in radians
    cosine_theta = np.clip(dot_product / (magnitude1 * magnitude2), -1.0, 1.0)
    angle_radians = math.acos(cosine_theta)

    # Convert to degrees
    angle_degrees = math.degrees(angle_radians)

    return angle_degrees

import numpy as np
import math

def get_back_angle(hip: np.ndarray, shoulder: np.ndarray) -> float:
    """Calculate the angle of the back relative to the vertical y-axis.

    Args:
        hip (np.ndarray): (x, y, confidence) of the hip keypoint.
        shoulder (np.ndarray): (x, y, confidence) of the shoulder keypoint.

    Returns:
        float: Back angle in degrees (0° = straight up, increasing as the back leans forward).
    """
    # Vector from hip to shoulder (back direction)
    vector1 = (shoulder[0] - hip[0], shoulder[1] - hip[1])
    #if (hip[0] > shoulder[0]):
     #   vector2 = (0, 1)
    #else:
    vector2 = (0, -1)
    

    # Compute dot product
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]

    # Compute magnitude of vector1
    magnitude1 = math.sqrt(vector1[0]**2 + vector1[1]**2)

    # Avoid division by zero
    if magnitude1 == 0:
        return 0

    # Calculate angle in radians
    cosine_theta = np.clip(dot_product / magnitude1, -1.0, 1.0)
    angle_radians = math.acos(cosine_theta)

    # Convert to degrees
    angle_degrees = math.degrees(angle_radians)

    return angle_degrees


def are_knees_bending(kps: np.ndarray) -> bool:
    """Check if the knees are bending based on MoveNet keypoints.

    Args:
        kps (np.ndarray): Array of (x, y, confidence) for all 17 MoveNet keypoints.

    Returns:
        bool: True if knees are bending (angle > 50 degrees), otherwise False.
    """
    global previousAngle, squat_reps, current_pos
    # MoveNet keypoint indices (from the MoveNet model)
    RIGHT_HIP = 12
    RIGHT_KNEE = 14
    RIGHT_ANKLE = 16
    LEFT_HIP = 11
    LEFT_KNEE = 13
    LEFT_ANKLE = 15

    right_hip = kps[RIGHT_HIP]
    right_knee = kps[RIGHT_KNEE]
    right_ankle = kps[RIGHT_ANKLE]

    left_hip = kps[LEFT_HIP]
    left_knee = kps[LEFT_KNEE]
    left_ankle = kps[LEFT_ANKLE]

    right_leg_angle = get_bending_angle(right_hip, right_knee, right_ankle)
    left_leg_angle = get_bending_angle(left_hip, left_knee, left_ankle)
    avg_angle = (left_leg_angle + right_leg_angle) / 2
    if avg_angle > 30 and previousAngle < 30 and current_pos =="up":
        
        current_pos = "squatting"
    
    if avg_angle < previousAngle and current_pos == "squatting":
        current_pos ="up"
        squat_reps +=1
        print(f"Reps: {squat_reps}")
        if previousAngle < 50:
            print("Bend knees all the way to 90 degrees")
        if previousAngle >= 50:
            print("Good depth!")
  
    previousAngle = avg_angle
        
    return avg_angle
def is_back_straight(kps):
    RIGHT_HIP = 12
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    LEFT_SHOULDER = 5

    right_hip = kps[RIGHT_HIP]
    right_shoulder = kps[RIGHT_SHOULDER]
    left_hip = kps[LEFT_HIP]
    left_shoulder = kps[LEFT_SHOULDER]

    right_back_angle = get_back_angle(right_hip, right_shoulder)
    left_back_angle = get_back_angle(left_hip, left_shoulder)

    avg_back_angle = (right_back_angle + left_back_angle) / 2
    if avg_back_angle > 145:  # Example threshold for bending too far forward
        print("Warning: Back is bent too far forward!")
    #print(avg_back_angle)
    return avg_back_angle

In [42]:
def eval_knees(kps):
    RIGHT_HIP = 12
    RIGHT_KNEE = 14
    
    LEFT_HIP = 11
    LEFT_KNEE = 13
    LEFT_ANKLE = 15

In [43]:
import cv2
import tensorflow as tf
import math
import numpy as np
import tensorflow_hub as hub
from collections import deque

# Load MoveNet model
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Constants for keypoint indices
HIP_LEFT = 5
HIP_RIGHT = 6

# Squat detection variables
previous_kps = None
current_pos = "up"
squat_reps = 0
threshold = 0.01  # Adjust based on movement sensitivity
alpha =0.6
hip_history = deque(maxlen=3)  # Store last 5 hip positions
min_hip = float('inf')
max_hip = float('-inf')
previousAngle = 0


def process_frame(frame):
    """ Preprocess the frame and run inference through MoveNet. """
    input_tensor = tf.image.resize(frame, (192, 192))
    input_tensor = tf.cast(input_tensor, dtype=tf.int32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    outputs = model.signatures["serving_default"](input_tensor)
    keypoints = outputs["output_0"].numpy().squeeze()  # Shape: (17, 3)
    return keypoints
def smooth_keypoints(new_kps, prev_kps, alpha=0.6):
    """ Applies exponential moving average for keypoint stabilization. """
    if prev_kps is None:
        return new_kps
    return alpha * new_kps + (1 - alpha) * prev_kps

def draw_skeleton(frame, keypoints, threshold=0.3):
    """
    Draws keypoints and connects them with lines to form a skeleton.
    Assumes keypoints are in normalized coordinates with shape (1, 1, 17, 3).
    """
    height, width, _ = frame.shape
    # Squeeze the output to shape (17, 3)
    #kpts = keypoints[0, 0, :, :]
    kpts = keypoints

    # Define skeleton connections (based on MoveNet keypoint indices):
    skeleton_connections = [
        (0, 1), (0, 2),       # Nose to eyes
        (1, 3), (2, 4),       # Eyes to ears
        (0, 5), (0, 6),       # Nose to shoulders
        (5, 7), (7, 9),       # Left arm (shoulder -> elbow -> wrist)
        (6, 8), (8, 10),      # Right arm (shoulder -> elbow -> wrist)
        (5, 6),               # Shoulders
        (5, 11), (6, 12),     # Shoulders to hips
        (11, 12),             # Hips
        (11, 13), (13, 15),    # Left leg (hip -> knee -> ankle)
        (12, 14), (14, 16)     # Right leg (hip -> knee -> ankle)
    ]

    # First, compute absolute positions for each keypoint
    abs_kpts = []
    for kp in kpts:
        y, x, conf = kp
        abs_kpts.append((int(x * width), int(y * height), conf))
        # Draw the keypoint if confidence is high
        if conf > threshold:
            cv2.circle(frame, (int(x * width), int(y * height)), 5, (0, 255, 0), -1)

    # Draw the skeleton connections
    for connection in skeleton_connections:
        i, j = connection
        x1, y1, conf1 = abs_kpts[i]
        x2, y2, conf2 = abs_kpts[j]
        # Only draw the connection if both keypoints are confidently detected
        if conf1 > threshold and conf2 > threshold:
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return frame



# Open video capture
video_path = "side-squat2.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0  # Frame counter

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process every 12th frame
    if frame_count % 6 == 0:
        keypoints = process_frame(frame)
        frame_with_skeleton = draw_skeleton(frame, keypoints)
        #detect_squat_repetition(keypoints)
        results= are_knees_bending(keypoints)
        backResults = is_back_straight(keypoints)
        print(backResults)
        
        

        
        
        #cv2.imshow("Pose Estimation - Exo Skeleton", frame_with_skeleton)
        

    cv2.imshow("Pose Estimation", frame)
    frame_count += 1  # Increment frame counter
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    

cap.release()
cv2.destroyAllWindows()


89.13206792134248
98.73182689165208
115.2942795577854
125.29339752438742
121.23226511502611
130.1730970823124
Reps: 1
Good depth!
128.5417527921696
131.74853576096905
139.7104787284706
135.58458760109914
134.06841856615807
133.88261005946597
129.9492339713946
122.39829714073272
122.33876493707956
106.88678608334342
88.60782597401666
82.23806556484155
83.68214270326128
81.86147617503855
86.51678314856136
104.51734438540475
121.4551086638912
127.46092678453178
133.83454195667628
127.42811652757436
131.25579495306806
Reps: 2
Good depth!
136.99284164075675
140.72900653882846
138.60544689324584
138.67035220887215
136.32178899435837
133.90848566649132
125.4649047728634
118.64985225571326
109.51673238912784
95.75689549638204
89.6650894517696
80.01267773457928
77.4478918461684
80.41845972491222
85.48095706568029
100.28823674314872
121.72832751290116
135.49449172625626
132.3446663112581
131.54939858563935
Reps: 3
Good depth!
127.96391772730988
124.21714554576684
122.40338290131069
122.384692465

KeyboardInterrupt: 